In [0]:
# Import the Pipeline Module
from pyspark.ml import Pipeline
# Modules that contains our Linear Regression & Random Forest Algorithms
from pyspark.ml.regression import LinearRegression , RandomForestRegressor
# Module with the Evaluator
from pyspark.ml.evaluation import RegressionEvaluator 
# One Hot Encode Module
from pyspark.ml.feature import VectorAssembler

In [0]:
# Store the database previous loaded
cars_temp = spark.read.format("csv").option("header", True ).option("inferSchema", True).load("/learn/Cars_Machine") 
cars_temp.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
 21| 6| 160| 39| 262| 1646| 0| 1| 4| 4|110|
 21| 6| 160| 39|2875| 1702| 0| 1| 4| 4|110|
 228| 4| 108| 385| 232| 1861| 1| 1| 4| 1| 93|
 214| 6| 258| 308|3215| 1944| 1| 0| 3| 1|110|
 187| 8| 360| 315| 344| 1702| 0| 0| 3| 2|175|
 181| 6| 225| 276| 346| 2022| 1| 0| 3| 1|105|
 143| 8| 360| 321| 357| 1584| 0| 0| 3| 4|245|
 244| 4| 1467| 369| 319| 20| 1| 0| 4| 2| 62|
 228| 4| 1408| 392| 315| 229| 1| 0| 4| 2| 95|
 192| 6| 1676| 392| 344| 183| 1| 0| 4| 4|123|
 178| 6| 1676| 392| 344| 189| 1| 0| 4| 4|123|
 164| 8| 2758| 307| 407| 174| 0| 0| 3| 3|180|
 173| 8| 2758| 307| 373| 176| 0| 0| 3| 3|180|
 152| 8| 2758| 307| 378| 18| 0| 0| 3| 3|180|
 104| 8| 472| 293| 525| 1798| 0| 0| 3| 4|205|
 104| 8| 460| 3|5424| 1782| 0| 0| 3| 4|215|
 147| 8| 440| 323|5345| 1742| 0| 0| 3| 4|230|
 324| 4| 787| 408| 22| 1947| 1| 1| 4| 1| 66|
 304| 4| 757| 493|1615| 1852| 1| 1| 4| 2| 52|
 339| 4| 711| 422|1835| 199| 1| 1| 4| 1| 65|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 20 rows

In [0]:
cars_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
 21| 6| 160| 39| 262| 1646| 0| 1| 4| 4|110|
 21| 6| 160| 39|2875| 1702| 0| 1| 4| 4|110|
 228| 4| 108| 385| 232| 1861| 1| 1| 4| 1| 93|
 214| 6| 258| 308|3215| 1944| 1| 0| 3| 1|110|
 187| 8| 360| 315| 344| 1702| 0| 0| 3| 2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows

In [0]:
# Store it in a shorter DF
cars = cars_temp.select("Consumo", "Cilindros" , "Cilindradas" , "HP")
cars.show(10)

+-------+---------+-----------+---+
Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
 21| 6| 160|110|
 21| 6| 160|110|
 228| 4| 108| 93|
 214| 6| 258|110|
 187| 8| 360|175|
 181| 6| 225|105|
 143| 8| 360|245|
 244| 4| 1467| 62|
 228| 4| 1408| 95|
 192| 6| 1676|123|
+-------+---------+-----------+---+
only showing top 10 rows

In [0]:
# Perform the One Hot Encode
vec_characteristics = VectorAssembler(inputCols=[("Consumo"),("Cilindros"), ("Cilindradas")] , outputCol= "characteristics")

In [0]:
# Transform our One Hot Encoded Data
Cars =  vec_characteristics.transform(cars)

In [0]:
Cars.show(7)

+-------+---------+-----------+---+-----------------+
Consumo|Cilindros|Cilindradas| HP| characteristics|
+-------+---------+-----------+---+-----------------+
 21| 6| 160|110| [21.0,6.0,160.0]|
 21| 6| 160|110| [21.0,6.0,160.0]|
 228| 4| 108| 93|[228.0,4.0,108.0]|
 214| 6| 258|110|[214.0,6.0,258.0]|
 187| 8| 360|175|[187.0,8.0,360.0]|
 181| 6| 225|105|[181.0,6.0,225.0]|
 143| 8| 360|245|[143.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 7 rows

In [0]:
# Defining the proportion of data to be tested and trained
cars_test , cars_train = Cars.randomSplit([0.75 , 0.25])

In [0]:
# Verify the proportion
print(cars_test.count())
print()
print(cars_train.count())

26

6

In [0]:
# Doing a Liear Regression of HP  using characteristics as feature
lin_reg = LinearRegression(featuresCol="characteristics" , labelCol="HP")

In [0]:
# Fitting the model
model = lin_reg.fit(cars_train)

In [0]:
# Using our Linear Regression to Predict the HP values (prediction)
predict  = model.transform(cars_test)
predict.show(8)

+-------+---------+-----------+---+-----------------+------------------+
Consumo|Cilindros|Cilindradas| HP| characteristics| prediction|
+-------+---------+-----------+---+-----------------+------------------+
 15| 8| 301|335| [15.0,8.0,301.0]|224.74479168846122|
 21| 6| 160|110| [21.0,6.0,160.0]|186.84780508236594|
 21| 6| 160|110| [21.0,6.0,160.0]|186.84780508236594|
 26| 4| 1203| 91|[26.0,4.0,1203.0]|143.09879031869426|
 104| 8| 460|215|[104.0,8.0,460.0]|203.84471178206906|
 133| 8| 350|245|[133.0,8.0,350.0]|197.86516871693505|
 143| 8| 360|245|[143.0,8.0,360.0]| 195.5572184901814|
 147| 8| 440|230|[147.0,8.0,440.0]| 194.2439164064849|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 8 rows

In [0]:
# We will Evaluate our model = > Comparing the HP (real value) with the predict one (preditions column)
evaluate = RegressionEvaluator(predictionCol="prediction" , labelCol="HP" , metricName="rmse" )

In [0]:
# Let´s see the result of the evaluation of our model
rmse = evaluate.evaluate(predict)
print(rmse)

43.21250420181425

In [0]:
# Create the pipeline object
pipelines = Pipeline(stages=[vec_characteristics , lin_reg])

In [0]:
# Fitting the pipeline model
model_pipeline = pipelines.fit(cars)

In [0]:
# Consulting the Pred Result
predict_pipeline = model_pipeline.transform(cars)
predict_pipeline.show()

+-------+---------+-----------+---+------------------+------------------+
Consumo|Cilindros|Cilindradas| HP| characteristics| prediction|
+-------+---------+-----------+---+------------------+------------------+
 21| 6| 160|110| [21.0,6.0,160.0]|162.32154816816646|
 21| 6| 160|110| [21.0,6.0,160.0]|162.32154816816646|
 228| 4| 108| 93| [228.0,4.0,108.0]| 82.51715587712931|
 214| 6| 258|110| [214.0,6.0,258.0]|141.86680518718754|
 187| 8| 360|175| [187.0,8.0,360.0]|202.93528239714834|
 181| 6| 225|105| [181.0,6.0,225.0]| 145.4980634611832|
 143| 8| 360|245| [143.0,8.0,360.0]| 207.41448530972|
 244| 4| 1467| 62|[244.0,4.0,1467.0]| 69.69282676584851|
 228| 4| 1408| 95|[228.0,4.0,1408.0]| 71.80767356085781|
 192| 6| 1676|123|[192.0,6.0,1676.0]|132.42483285541724|
 178| 6| 1676|123|[178.0,6.0,1676.0]|133.85003378214458|
 164| 8| 2758|180|[164.0,8.0,2758.0]|185.52180807776818|
 173| 8| 2758|180|[173.0,8.0,2758.0]|184.60560748201488|
 152| 8| 2758|180|[152.0,8.0,2758.0]| 186.7434088721059|
 104| 8| 472|205| [104.0,8.0,472.0]| 210.4620247994542|
 104| 8| 460|215| [104.0,8.0,460.0]|210.56088155929672|
 147| 8| 440|230| [147.0,8.0,440.0]|206.34823997932406|
 324| 4| 787| 66| [324.0,4.0,787.0]| 67.15070452800569|
 304| 4| 757| 52| [304.0,4.0,757.0]| 69.43384775150815|
 339| 4| 711| 65| [339.0,4.0,711.0]| 66.24979634741939|
+-------+---------+-----------+---+------------------+------------------+
only showing top 20 rows

In [0]:
# Let´s verify the action of the Pipeline Model to caompare with the other ones
rmse_pipeline = evaluate.evaluate(predict_pipeline)
print(rmse_pipeline)

35.96319777272619

In [0]:
# Apply the Random Forest Algorithm
forest = RandomForestRegressor(featuresCol="characteristics" , labelCol="HP")

In [0]:
# Creating the train model
model_forest = forest.fit(cars_train)

In [0]:
# Set the prection model
predict_forest = model_forest.transform(cars_test)
predict_forest.show()

+-------+---------+-----------+---+------------------+------------------+
Consumo|Cilindros|Cilindradas| HP| characteristics| prediction|
+-------+---------+-----------+---+------------------+------------------+
 15| 8| 301|335| [15.0,8.0,301.0]|163.79333333333332|
 21| 6| 160|110| [21.0,6.0,160.0]|140.87083333333334|
 21| 6| 160|110| [21.0,6.0,160.0]|140.87083333333334|
 26| 4| 1203| 91| [26.0,4.0,1203.0]| 142.9308333333333|
 104| 8| 460|215| [104.0,8.0,460.0]|163.79333333333332|
 133| 8| 350|245| [133.0,8.0,350.0]|150.64333333333332|
 143| 8| 360|245| [143.0,8.0,360.0]|150.64333333333332|
 147| 8| 440|230| [147.0,8.0,440.0]|150.64333333333332|
 152| 8| 304|150| [152.0,8.0,304.0]|150.64333333333332|
 155| 8| 318|150| [155.0,8.0,318.0]|150.64333333333332|
 158| 8| 351|264| [158.0,8.0,351.0]|150.64333333333332|
 164| 8| 2758|180|[164.0,8.0,2758.0]|161.44333333333333|
 173| 8| 2758|180|[173.0,8.0,2758.0]|161.44333333333333|
 178| 6| 1676|123|[178.0,6.0,1676.0]|128.98083333333332|
 181| 6| 225|105| [181.0,6.0,225.0]|128.77083333333331|
 187| 8| 360|175| [187.0,8.0,360.0]| 117.535|
 192| 6| 1676|123|[192.0,6.0,1676.0]| 101.1725|
 192| 8| 400|175| [192.0,8.0,400.0]| 117.535|
 197| 6| 145|175| [197.0,6.0,145.0]| 105.3125|
 214| 6| 258|110| [214.0,6.0,258.0]| 105.3125|
+-------+---------+-----------+---+------------------+------------------+
only showing top 20 rows

In [0]:
# Evaluating our Random Forest Prediction
rmse_forest = evaluate.evaluate(predict_forest)
print(rmse_forest)

58.49536161345155